In [ ]:
import pandas as pd
raw_data = pd.read_csv('raw_total_fight_data.csv',sep = ";")

raw_data.dropna(how="any", axis = 0, inplace = True)

B_WINS = []

for index, row in raw_data.iterrows():
    blue_fighter = row['B_fighter']
    red_fighter = row['R_fighter']
    date = row['date']
    blue_wins = 0
    blue_losses = 0
    
    print(blue_fighter)
    
    #iterate through all past fights
    for index, row in raw_data.iterrows():
        #check that it is a past fight
        if (row['date'] < date):
            #update blue fighters historical data
            if (blue_fighter == row['B_fighter']):
                blue_wins += 1
    
    B_WINS.append(blue_wins)

raw_data['B_WINS'] = B_WINS         

print(raw_data)

Marlon Moraes
Jessica Eye
Donald Cerrone
Petr Yan
Blagoy Ivanov
Nina Ansaroff
Pedro Munhoz
Alexa Grasso
Calvin Kattar
Angela Hill
Darren Stewart
Grigorii Popov
Joanne Calderwood
Anthony Smith
Aleksandar Rakic
Chris Fishgold
Christos Giagos
Sung Bin Jo
Sergey Khandozhko
Lina Lansberg
Leonardo Santos
Frank Camacho
Devin Clark
Danilo Belluardo
Kevin Lee
Ian Heinisch
Felicia Spencer
Derrick Krantz
Nik Lentz
Austin Hubbard
Sijara Eubanks
Charles Jourdain
Michel Pereira
Grant Dawson
Ed Herman
Trevin Giles
Julian Erosa
Jessica Andrade
Anderson Silva
Alexander Volkanovski
Laureano Staropoli
Bethe Correia
Ryan Spann
Kurt Holobaugh
Sergio Moraes
Clay Guida
Priscila Cachoeira
Carlos Huachin
Viviane Araujo
Donald Cerrone
Elias Theodorou
Shane Burgos
Merab Dvalishvili
Serghei Spivac
Andrew Sanchez
Sarah Moras
Vince Morales
Kyle Prepolec
Matt Sayles
Juan Adams
Cole Smith
Jack Hermansson
Dmitrii Smoliakov
Mike Perry
Ion Cutelaba
Cory Sandhagen
Thomas Gifford
Takashi Sato
Augusto Sakai
Virna Jandiroba